In [1]:
import cv2
import numpy as np
from pathlib import Path
import os
from tqdm import tqdm
import matplotlib.pyplot as plt
from rembg import remove
from PIL import Image
from rembg import new_session
import os
from pathlib import Path

In [18]:
# Configuration
INPUT_DIR = "character_frames"  # Directory containing character frames
OUTPUT_DIR = "character_frames_no_bg"  # Directory to save frames with transparent backgrounds

# Create output directory
os.makedirs(OUTPUT_DIR, exist_ok=True)

In [3]:
# Download and initialize rembg model for offline use
print("Downloading and initializing rembg model for offline use...")
print("This will download ~100MB of model data (one-time setup)")

# Set up local model directory
local_model_dir = Path("./rembg_models")
local_model_dir.mkdir(exist_ok=True)

# Set environment variable to use local directory
os.environ['U2NET_HOME'] = str(local_model_dir)

This will download ~100MB of model data (one-time setup)


In [ ]:
try:
    # Create a session with the default u2net model
    # This will download the model to the local directory
    rembg_session = new_session('u2net')
    print("✅ Model downloaded and initialized successfully!")
    print(f"Model saved locally in: {local_model_dir}")
    print("You can now run background removal offline")
    model_ready = True
    
    # Verify the model files exist locally
    model_files = list(local_model_dir.glob("*.pth"))
    if model_files:
        print(f"Model file(s) found: {[f.name for f in model_files]}")
    
except Exception as e:
    print(f"❌ Error downloading model: {e}")
    print("Falling back to color-based methods")
    model_ready = False

In [4]:
# Load rembg model from local directory
print("Loading rembg model from local directory...")

try:
    # Check if model directory exists
    if local_model_dir.exists():
        model_files = list(local_model_dir.glob("*.onnx"))
        if model_files:
            print(f"Found local model files: {[f.name for f in model_files]}")
            
            # Create session with local model
            rembg_session = new_session('u2net')
            model_ready = True
            print("✅ Model loaded successfully from local directory!")
            
        else:
            print("❌ No model files found in local directory")
            print("Please run the model download cell first")
            model_ready = False
    else:
        print("❌ Local model directory does not exist")
        print("Please run the model download cell first")
        model_ready = False
        
except Exception as e:
    print(f"❌ Error loading local model: {e}")
    model_ready = False

Loading rembg model from local directory...
Found local model files: ['u2net.onnx']
✅ Model loaded successfully from local directory!
✅ Model loaded successfully from local directory!


In [5]:
def remove_background_rembg(input_path, output_path, session=None):
    """
    Remove background from an image using rembg library
    
    Args:
        input_path: Path to input image
        output_path: Path to save output image with transparent background
        session: Pre-initialized rembg session (optional, will create new if None)
    
    Returns:
        bool: True if successful, False otherwise
    """
    try:
        # Read input image
        with open(input_path, 'rb') as input_file:
            input_data = input_file.read()
        
        # Remove background using session if provided, otherwise use default
        if session is not None:
            output_data = remove(input_data, session=session)
        else:
            output_data = remove(input_data)
        
        # Save output image
        with open(output_path, 'wb') as output_file:
            output_file.write(output_data)
        
        return True
    except Exception as e:
        print(f"Error processing {input_path}: {e}")
        return False

In [6]:
def remove_background_color_mask(input_path, output_path, method='adaptive'):
    """
    Remove background using color-based masking (alternative method)
    
    Args:
        input_path: Path to input image
        output_path: Path to save output image with transparent background
        method: 'adaptive' for adaptive thresholding, 'hsv' for HSV-based masking
    
    Returns:
        bool: True if successful, False otherwise
    """
    try:
        # Read image
        img = cv2.imread(str(input_path))
        if img is None:
            print(f"Could not read image: {input_path}")
            return False
        
        if method == 'adaptive':
            # Convert to grayscale
            gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            
            # Apply adaptive thresholding to separate foreground/background
            thresh = cv2.adaptiveThreshold(gray, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, 
                                         cv2.THRESH_BINARY, 11, 2)
            
            # Create mask (invert so character is white, background is black)
            mask = cv2.bitwise_not(thresh)
            
            # Apply morphological operations to clean up the mask
            kernel = np.ones((3,3), np.uint8)
            mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel)
            mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, kernel)
            
        elif method == 'hsv':
            # Convert to HSV
            hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
            
            # Define range for background color (you may need to adjust these values)
            # This assumes a light background
            lower_bg = np.array([0, 0, 180])
            upper_bg = np.array([180, 30, 255])
            
            # Create mask for background
            bg_mask = cv2.inRange(hsv, lower_bg, upper_bg)
            
            # Invert mask so character is white, background is black
            mask = cv2.bitwise_not(bg_mask)
        
        # Create 4-channel image (BGRA)
        b, g, r = cv2.split(img)
        
        # Use mask as alpha channel
        rgba = cv2.merge([b, g, r, mask])
        
        # Save as PNG to preserve transparency
        cv2.imwrite(str(output_path), rgba)
        
        return True
    except Exception as e:
        print(f"Error processing {input_path}: {e}")
        return False

In [7]:
def process_all_character_frames(method='rembg'):
    """
    Process all character frames to remove backgrounds
    
    Args:
        method: 'rembg' for AI-based removal, 'adaptive' or 'hsv' for color-based methods
    
    Returns:
        dict: Processing results
    """
    # Check if using rembg but model not ready
    if method == 'rembg' and not globals().get('model_ready', False):
        print("⚠️  rembg model not ready! Please run the model download cell first.")
        print("Falling back to adaptive color-based method...")
        method = 'adaptive'
    
    input_dir = Path(INPUT_DIR)
    
    if not input_dir.exists():
        print(f"Input directory {INPUT_DIR} does not exist!")
        return {}
    
    # Find all character directories
    character_dirs = [d for d in input_dir.iterdir() if d.is_dir()]
    
    if not character_dirs:
        print(f"No character directories found in {INPUT_DIR}")
        return {}
    
    print(f"Found {len(character_dirs)} character directories")
    print(f"Using method: {method}")
    
    # Get the rembg session if available
    session = globals().get('rembg_session', None) if method == 'rembg' else None
    
    results = {}
    total_processed = 0
    total_failed = 0
    
    for char_dir in character_dirs:
        character_id = char_dir.name
        print(f"\nProcessing character {character_id}...")
        
        # Create output directory for this character
        output_char_dir = Path(OUTPUT_DIR) / character_id
        output_char_dir.mkdir(exist_ok=True)
        
        # Check if output directory already exists and has files
        if output_char_dir.exists() and list(output_char_dir.glob("*.png")):
            print(f"  Character {character_id} already processed, skipping...")
            continue
        
        # Find all frame files for this character
        frame_files = list(char_dir.glob(f"{character_id}_*.jpg"))
        
        if not frame_files:
            print(f"  No frames found for character {character_id}")
            continue
        
        char_results = {
            'character_id': character_id,
            'total_frames': len(frame_files),
            'processed': 0,
            'failed': 0,
            'output_dir': str(output_char_dir)
        }
        
        # Process each frame
        for frame_file in tqdm(frame_files, desc=f"Processing {character_id}"):
            output_filename = frame_file.stem + '.png'
            output_path = output_char_dir / output_filename
            
            # Skip if already processed
            if output_path.exists():
                char_results['processed'] += 1
                continue
            
            # Remove background
            if method == 'rembg':
                success = remove_background_rembg(frame_file, output_path, session)
            else:
                success = remove_background_color_mask(frame_file, output_path, method)
            
            if success:
                char_results['processed'] += 1
                total_processed += 1
            else:
                char_results['failed'] += 1
                total_failed += 1
        
        results[character_id] = char_results
        print(f"  Processed: {char_results['processed']}, Failed: {char_results['failed']}")
    
    # Print summary
    print(f"\n=== BACKGROUND REMOVAL COMPLETE ===")
    print(f"Total characters processed: {len(results)}")
    print(f"Total frames processed: {total_processed}")
    print(f"Total failures: {total_failed}")
    print(f"Output saved to: {OUTPUT_DIR}")
    
    return results

In [15]:
def visualize_before_after(character_id, frame_num=1):
    """
    Visualize original and background-removed frames side by side
    
    Args:
        character_id: ID of the character to visualize
        frame_num: Frame number to display (1-8)
    """
    # Construct file paths
    original_path = Path(INPUT_DIR) / character_id / f"{character_id}_{frame_num}.jpg"
    processed_path = Path(OUTPUT_DIR) / character_id / f"{character_id}_{frame_num}.png"
    
    if not original_path.exists():
        print(f"Original frame not found: {original_path}")
        return
    
    if not processed_path.exists():
        print(f"Processed frame not found: {processed_path}")
        return
    
    # Read images
    original = cv2.imread(str(original_path))
    original_rgb = cv2.cvtColor(original, cv2.COLOR_BGR2RGB)
    
    # Read PNG with transparency
    processed = cv2.imread(str(processed_path), cv2.IMREAD_UNCHANGED)
    
    # Create visualization
    fig, axes = plt.subplots(1, 3, figsize=(15, 5))
    
    # Original image
    axes[0].imshow(original_rgb)
    axes[0].set_title("Original")
    axes[0].axis('off')
    
    # Processed image on white background
    if processed.shape[2] == 4:  # Has alpha channel
        # Create white background
        white_bg = np.ones((processed.shape[0], processed.shape[1], 3), dtype=np.uint8) * 255
        
        # Convert BGRA to RGBA
        processed_rgba = cv2.cvtColor(processed, cv2.COLOR_BGRA2RGBA)
        
        # Blend with white background
        alpha = processed_rgba[:, :, 3:4] / 255.0
        processed_rgb = processed_rgba[:, :, :3]
        blended = (processed_rgb * alpha + white_bg * (1 - alpha)).astype(np.uint8)
        
        axes[1].imshow(blended)
        axes[1].set_title("Removed BG (on white)")
    else:
        processed_rgb = cv2.cvtColor(processed, cv2.COLOR_BGR2RGB)
        axes[1].imshow(processed_rgb)
        axes[1].set_title("Processed")
    axes[1].axis('off')
    
    # Processed image on checkered background to show transparency
    if processed.shape[2] == 4:
        # Create checkered background
        h, w = processed.shape[:2]
        checkered = np.zeros((h, w, 3), dtype=np.uint8)
        for i in range(0, h, 20):
            for j in range(0, w, 20):
                if (i//20 + j//20) % 2 == 0:
                    checkered[i:i+20, j:j+20] = [200, 200, 200]
                else:
                    checkered[i:i+20, j:j+20] = [255, 255, 255]
        
        # Blend with checkered background
        alpha = processed_rgba[:, :, 3:4] / 255.0
        processed_rgb = processed_rgba[:, :, :3]
        blended_check = (processed_rgb * alpha + checkered * (1 - alpha)).astype(np.uint8)
        
        axes[2].imshow(blended_check)
        axes[2].set_title("Transparency Preview")
    else:
        axes[2].imshow(processed_rgb)
        axes[2].set_title("Processed")
    axes[2].axis('off')
    
    plt.suptitle(f"Character {character_id} - Frame {frame_num}", fontsize=16)
    plt.tight_layout()
    plt.show()

In [ ]:
# Check if model is ready and run background removal
print("=== BACKGROUND REMOVAL EXECUTION ===")

# Check if rembg model is ready
if globals().get('model_ready', False):
    print("✅ rembg model is ready! Using AI-based background removal...")
    results_rembg = process_all_character_frames(method='rembg')
else:
    print("⚠️  rembg model not downloaded yet.")

=== BACKGROUND REMOVAL EXECUTION ===
✅ rembg model is ready! Using AI-based background removal...
Found 2338 character directories
Using method: rembg

Processing character 10634...


Processing 10634:   0%|          | 0/8 [00:00<?, ?it/s]

Processing 10634: 100%|██████████| 8/8 [00:04<00:00,  1.72it/s]


  Processed: 8, Failed: 0

Processing character 869...


Processing 869: 100%|██████████| 8/8 [00:04<00:00,  1.80it/s]


...

Processing 865: 100%|██████████| 8/8 [00:04<00:00,  1.91it/s]


  Processed: 8, Failed: 0

Processing character 780...


Processing 780: 100%|██████████| 8/8 [00:04<00:00,  1.91it/s]


  Processed: 8, Failed: 0

Processing character 3272...


Processing 3272:  75%|███████▌  | 6/8 [00:03<00:01,  1.59it/s]

In [16]:
# Visualize results for a specific character
# Replace with an actual character ID from your data
if results_rembg:
    # Get first character from results
    first_character = '1'
    print(f"Visualizing results for character: {first_character}")
    
    # Show before/after for frame 1
    visualize_before_after(first_character, frame_num=1)
    
    # Show before/after for frame 5
    visualize_before_after(first_character, frame_num=5)